In [ ]:
!pip install easyocr -qq

In [ ]:
import easyocr
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import clear_output


In [ ]:
from google.colab import files

uploaded = files.upload()

# Obtén el nombre del archivo cargado (asumiendo que solo se cargó un archivo)
imagen = list(uploaded.keys())[0]

In [ ]:
# Reemplaza 'nombre_imagen.jpg' con el nombre de tu archivo de imagen
imagen = cv2.imread('cedula.png')
cv2_imshow(imagen)


In [ ]:
# Puedes aplicar técnicas de preprocesamiento aquí si es necesario
# Por ejemplo, convertir a escala de grises, corrección de perspectiva, etc.
# En este ejemplo, simplemente redimensionamos la imagen
scale_percent = 50 # porcentaje de la escala original
width = int(imagen.shape[1] * scale_percent / 100)
height = int(imagen.shape[0] * scale_percent / 100)
dim = (width, height)
imagen_resized = cv2.resize(imagen, dim, interpolation=cv2.INTER_AREA)

In [ ]:
lector = easyocr.Reader(['es'], gpu=True) # Reemplaza 'en' con el código de idioma de la identificación si es necesario

resultado = lector.readtext(imagen_resized)

# Imprimir y mostrar el resultado
clear_output()
cv2_imshow(imagen_resized)
for i, r in enumerate(resultado):
    print(f"Texto {i + 1}: {r[1]}")


In [ ]:
imagen_con_bloques = imagen_resized.copy()

for i, r in enumerate(resultado):
    (x1, y1), (x2, y2), (x3, y3), (x4, y4) = r[0]
    cv2.rectangle(imagen_con_bloques, (int(x1), int(y1)), (int(x3), int(y3)), (0, 255, 0), 2)
    
    bloque_numero = f"{i + 1}"
    cv2.putText(imagen_con_bloques, bloque_numero, (int(x1), int(y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2_imshow(imagen_con_bloques)


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Carga la imagen del DNI
img = cv2.imread('cedula.png')

# Convierte la imagen a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplica umbralización adaptativa
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Aplica dilatación y erosión para eliminar el ruido y conectar regiones
kernel = np.ones((3, 3), np.uint8)
thresh = cv2.dilate(thresh, kernel, iterations=1)
thresh = cv2.erode(thresh, kernel, iterations=1)

# Encuentra los contornos en la imagen y selecciona el más grande
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]

# Calcula los puntos de la imagen para aplicar la transformación de perspectiva
rect = cv2.minAreaRect(biggest_contour)
box = cv2.boxPoints(rect)
box = np.int0(box)

# Calcula la matriz de transformación y aplica la transformación a la imagen
width, height = 600, 400
src_pts = box.astype("float32")
dst_pts = np.array([[0, height-1], [0, 0], [width-1, 0], [width-1, height-1]], dtype="float32")
M = cv2.getPerspectiveTransform(src_pts, dst_pts)
dst = cv2.warpPerspective(img, M, (width, height))

# Muestra la imagen original y la imagen transformada
cv2_imshow(img)
cv2_imshow(dst)


In [ ]:
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev

In [ ]:
!pip install pytesseract


In [ ]:
import cv2
import pytesseract
from pytesseract import Output


In [ ]:
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    return thresh


In [ ]:
def extract_info(image, field_coords):
    # Apply OCR on the image
    data = pytesseract.image_to_data(image, output_type=Output.DICT)

    extracted_info = {}

    for name, coords in field_coords.items():
        x, y, w, h = coords
        field_text = ''

        for i in range(len(data['text'])):
            if (
                data['left'][i] >= x and data['top'][i] >= y
                and data['left'][i] + data['width'][i] <= x + w
                and data['top'][i] + data['height'][i] <= y + h
            ):
                field_text += data['text'][i] + ' '

        extracted_info[name] = field_text.strip()

    return extracted_info


In [ ]:
# Define field coordinates (replace these with the correct coordinates for your ID cards)
field_coords = {
    'Name': (50, 50, 200, 40),
    'ID Number': (50, 100, 200, 40),
    'Date of Birth': (50, 150, 200, 40)
}

# Process each ID card image (replace 'id_card_1.jpg', 'id_card_2.jpg' with your file names)
for image_path in ['cedula.png']:
    preprocessed_image = preprocess_image(image_path)
    extracted_info = extract_info(preprocessed_image, field_coords)

    print(f"Information extracted from {image_path}:")
    for field, value in extracted_info.items():
        print(f"{field}: {value}")

    print("\n")
